In [1]:
import cv2
import mediapipe as mp
import numpy as np

In [2]:
# Initialize Mediapipe Pose
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)

# Video Pose Detection

In [3]:
def calculate_angle(a, b, c):
    """
    Calculate the angle between three points.
    """
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)

    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)

    if angle > 180.0:
        angle = 360.0 - angle

    return angle

In [4]:
# Function to determine current stage (up or down)
def current_stage(angle):
    if angle >= 130:
        return "Up"
    elif angle <= 90:
        return "Down"
    else:
        return "Mid"

In [5]:
# ankles, knees, hip, shoulder
def posture_check(shoulder, hip, knee, ankle):
    """
    Check if the body is in a relatively straight line based on the angles of the hip and knee.
    """
    hip_angle = calculate_angle(shoulder, hip, knee)
    knee_angle = calculate_angle(hip, knee, ankle)

    if 160 <= hip_angle <= 200 and 160 <= knee_angle <= 200:
        return "Straight", hip_angle, knee_angle
    else:
        return "Bent", hip_angle, knee_angle

In [6]:
# def count_pushups(landmarks, counter, stage):
#     """
#     Count the number of push-ups based on elbow angle.
#     """
#     shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
#                 landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
#     elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
#              landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
#     wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
#              landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

#     elbow_angle = calculate_angle(shoulder, elbow, wrist)

#     if elbow_angle > 160:
#         stage = "Up"
#     if elbow_angle < 90 and stage == 'Up':
#         stage = "Down"
#         counter += 1
#         print(counter)

In [7]:
# Initialize variables for counting push-ups and posture checking
counter = 0
# incorrect_count = 0
correct_posture = True

# Capture video feed
cap = cv2.VideoCapture(0)

# Change video resolution
width = 1280  # Desired width
height = 720  # Desired height
cap.set(cv2.CAP_PROP_FRAME_WIDTH, width)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, height)


while cap.isOpened():
    ret, frame = cap.read()
    
    # Recolor image to RGB
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    
    # Make pose detection
    results = pose.process(image)
    
    # Recolor back to BGR
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    
    # Extract landmarks
    try:
        landmarks = results.pose_landmarks.landmark
        
        # Get coordinates of relevant landmarks
        shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                    landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
        elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                    landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
        wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                    landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
        
        hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
        knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
        ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                    landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
        
        # Calculate elbow angle
        elbow_angle = calculate_angle(shoulder, elbow, wrist)
        
        # Determine current stage (up or down)
        stage = current_stage(elbow_angle)
        
        # Check if body is in a straight line
        posture_status = posture, hip_angle, knee_angle = posture_check(shoulder, hip, knee, ankle)

        
        previous_stage = ''
        if (stage == 'Down'):
            counter +=1

        
        # # Check posture and count reps
        # if stage == "Down":
        #     if elbow_angle >= 90:
        #         incorrect_count = 0
        #         correct_posture = True
        #     else:
        #         incorrect_count += 1
        #         if incorrect_count == 3:
        #             cv2.putText(image, "Fix your posture! Must bend your elbows at least 90 degrees!", 
        #                         (50, 100), 
        #                         cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2, cv2.LINE_AA)
        #             correct_posture = False
        # elif stage == "Up":
        #     if correct_posture:
        #         counter += 1
        #         correct_posture = False
        
        # Render push-up counter and feedback
        cv2.putText(image, 'Reps: {}'.format(counter), 
                    (50, 50), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
        cv2.putText(image, 'Stage: {}'.format(stage), 
                    (50, 100), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
        cv2.putText(image, 'Posture: {}'.format(posture), 
                    (50, 150), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        cv2.putText(image, 'Elbow Angle: {:.2f}'.format(elbow_angle), 
                    (50, 200), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2, cv2.LINE_AA)
        cv2.putText(image, 'Hip Angle: {:.2f}'.format(hip_angle), 
                    (50, 250), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2, cv2.LINE_AA)
        cv2.putText(image, 'Knee Angle: {:.2f}'.format(knee_angle), 
                    (50, 300), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2, cv2.LINE_AA)
        cv2.putText(image, 'Reps: {}'.format(posture_status), 
                    (50, 50), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                    mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                    mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2))
    
    except:
        pass
    
    # Display the output
    cv2.imshow('Push-Up Tracker', image)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()

c:\Users\alimo\anaconda3\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
